# importing libraries

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

## inputing python code to clean data

In [34]:
import return_clean_text as ct

## inputing training data and test data

In [37]:
data=pd.read_csv('./Train/Train.csv')                        #loading training data
x_test=pd.read_csv('./Test/Test.csv').values[:,0].tolist()   #loading test data and values[:,0] is flatting 2d list into 1d as our cleaning function will not work

In [38]:
print(x_test[0])

Remember those old kung fu movies we used to watch on Friday and Saturday late nights when our babysitters THOUGHT we were in charge? Well, this movie plays exactly like one of those movies. Patsy Kensit's biggest claim to fame was the love interest to Mel Gibson's character in "Lethal Weapon 2," and this performance was one of the reasons why she's never made it big: she's a terrible actress.<br /><br />In "Lethal Weapon 2," I thought she was cute. Cute enough to check out some of the other movies she'd been in, including "Loves Music, Loves to Dance" another big let down, which I, obviously, was not impressed with, either. But, as attractive as she is to my eyes, my soul screamed at me to turn it off because she played another cheap, predictable role, and done it very badly.<br /><br />In this movie, Kensit stars as a comedienne (and not a good one, either) who's working the clubs of France (couldn't cut it in her own homeland, so she's making THEIR ears bleed), who's down on her luc

In [39]:
data.head(7)

review label
0  mature intelligent and highly charged melodram...   pos
1  http://video.google.com/videoplay?docid=211772...   pos
2  Title: Opera (1987) Director: Dario Argento Ca...   pos
3  I think a lot of people just wrote this off as...   pos
4  This is a story of two dogs and a cat looking ...   pos
5  Steve Carell comes into his own in his first s...   pos
6  I'm only going to write more because it's requ...   neg

In [40]:
rev={'pos' : 1,'neg' : 0}
data.label=[rev[item] for item in data.label]

In [41]:
print(data.head(7))

                                              review  label
0  mature intelligent and highly charged melodram...      1
1  http://video.google.com/videoplay?docid=211772...      1
2  Title: Opera (1987) Director: Dario Argento Ca...      1
3  I think a lot of people just wrote this off as...      1
4  This is a story of two dogs and a cat looking ...      1
5  Steve Carell comes into his own in his first s...      1
6  I'm only going to write more because it's requ...      0


In [42]:
x1=data[data['label']==1].values[:,0].tolist()
x0=data[data['label']==0].values[:,0].tolist()

In [43]:
x1_clean=[ct.getCleanReview(i) for i in x1]          #cleaning training x1
x0_clean=[ct.getCleanReview(j) for j in x0]          #cleaning training x0
x_test_clean=[ct.getCleanReview(k) for k in x_test]  #cleaning training x_test

In [44]:
print(x1_clean[1])
print(x_test_clean[0])

http video googl com videoplay docid 211772166650071408 hl en distribut tri opt mass appeal want best possibl view rang forgo profit continu manual labor job gladli entertain work view texa tale pleas write like not like alex not like stuie texa texa tale write opinion rule
rememb old kung fu movi use watch friday saturday late night babysitt thought charg well movi play exactli like one movi patsi kensit biggest claim fame love interest mel gibson charact lethal weapon 2 perform one reason never made big terribl actress lethal weapon 2 thought cute cute enough check movi includ love music love danc anoth big let obvious not impress either attract eye soul scream turn play anoth cheap predict role done badli movi kensit star comedienn not good one either work club franc cut homeland make ear bleed luck even wors french govern want throw expir visa mayb caught act get marri casanova freiss luck predict begin terribl way give movi neg rate 1 10 star rate


In [45]:
cv1=CountVectorizer()
cv0=CountVectorizer()
vectorized_corpus1=cv1.fit_transform(x1_clean)
vectorized_corpus0=cv0.fit_transform(x0_clean)
vectorized_corpus1 = np.sum(vectorized_corpus1.toarray(),axis=0)
vectorized_corpus0 = np.sum(vectorized_corpus0.toarray(),axis=0)

In [46]:
a1=cv1.vocabulary_.keys()
a0=cv0.vocabulary_.keys()
vocab1=[]
vocab0=[]
for i in a1:
    vocab1.append(i)
for j in a0:
    vocab0.append(j)

In [47]:
def predict_single_line_review(vocab1,vocab0,x1_clean,x0_clean,x_test,cv1,cv0,vectorized_corpus1,vectorized_corpus0):
    v=len(vocab1)+len(vocab0)
    total=len(x1_clean)+len(x0_clean)
    p1=predict_posterior(vocab1,x1_clean,x_test,cv1,v,total,vectorized_corpus1)
    p0=predict_posterior(vocab0,x0_clean,x_test,cv0,v,total,vectorized_corpus0)
    if p1>p0:
        return 1
    else:
        return 0

def predict_posterior(vocab,x_clean,x_test,cv,v,total,vectorized_corpus):
    p=1.0
    x_test=x_test.split()
    for i in x_test:
        p*=likelihood(vocab,i,cv,v,vectorized_corpus)
    
    prior=len(x_clean)/float(total)
    p=p*prior
    
    return p

def likelihood(vocab,review,cv,v,vectorized_corpus):
    if review in vocab:
        index=cv.vocabulary_[review]
        frequency=vectorized_corpus[index]
        value=(frequency+1)/float(len(vocab)+v)
        return value
    else:
        return (1/float(v))

In [48]:
y_predicted=[]
for z in x_test_clean:
    y=predict_single_line_review(vocab1,vocab0,x1_clean,x0_clean,z,cv1,cv0,vectorized_corpus1,vectorized_corpus0)
    y_predicted.append(y)


In [54]:
y_predicted[1]

0

In [50]:
len(y_predicted)

10000

In [55]:
y_predicted=np.array(y_predicted)

In [56]:
y_predicted.shape

(10000,)

In [58]:
df=pd.DataFrame(data=y_predicted,columns=['label'])

In [60]:
rev1={1: 'pos' ,0: 'neg' }
df.label=[rev1[item] for item in df.label]

In [61]:
df.head(5)

label
0   neg
1   neg
2   neg
3   pos
4   pos

In [63]:
df.to_csv('y_prediction_myCode.csv',index_label='Id')